<a href="https://colab.research.google.com/github/sugarcane-mk/KALDI-ASR/blob/main/dataprep_4_asr_kaldi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pydub

# Segmentation



The audio files are segmented into individual sentences using `.lab` files, which contain the start and end timestamps for each sentence. These timestamps guide the extraction of specific segments from the audio files, creating individual files for each sentence.






In [ ]:
# !pip install pydub
from pydub import AudioSegment
import IPython.display as ipd
with open('/content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/label/478-487/478-487(10).lab','r') as f:  #lab file
  lines = f.readlines()

lines=lines[1::2]

# lines=lines[16::2]
# lines=['66275000 85800000 61\n',
#  '104650000 122900057 62\n',
#  '137975000 151775000 63\n',
#  '161975000 175725000 64\n',
#  '187150000 207500000 65\n',
#  '216250000 235299943 66\n',
#  '248850000 265975000 67\n',
#  '278700000 298274943 68\n',
#  '320200000 342550000 69\n',
# '383950000 407500057 70\n']

print('No. of sentences : ', len(lines))

lines


In [ ]:
from pydub import AudioSegment
import IPython.display as ipd
import os
import re

audio_file = '/content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/label/478-487/478-487(10).wav'   # Audio files path

match = re.search(r"/Segmented_data/([^/]+)/label/([^/]+)/", audio_file)  # re search to extract speaker and sentence folder name

# Speaker and session information
speaker_folder = match.group(1)      # speaker folder name
sentence_folder = match.group(2)     # sentence folder name
speaker = "FPU"                      # speaker name
session = 10                          # session number


for line in lines:
  parts = line.strip().split()
  start_time = float(parts[0]) / 10000
  end_time = float(parts[1]) / 10000
  sentence_number = int(parts[2])

  audio = AudioSegment.from_file(audio_file)
  extracted_audio = audio[start_time:end_time]

  output_file = f"/content/drive/Shareddrives/Audio_ processing/Segmented_data/{speaker_folder}/speech/{sentence_folder}/{speaker}_{sentence_number}_{session}.wav"   # MSU_1_1.wav # Audio file path
  os.makedirs(os.path.dirname(output_file), exist_ok=True)
  extracted_audio.export(output_file, format="wav")
  # print(f'File saved to path {output_file}')


print(f"Process completed for speaker {speaker_folder} for sentence {sentence_folder}, and session {session}.")

MSR
*   131-155(7).....142 missing
*   131-155(9).....152 missing
*   156-180(5).....161 162 missing
*   156-180(8).....159-162 164 176 missing
*   156-180(9).....160-162 168 176 missing
*   181-196-all....186 number issue

FPU
*  262-285(10).....280 Missing

In [ ]:
from pydub import AudioSegment
import IPython.display as ipd
import os

audio_file = '/content/drive/Shareddrives/Audio_ processing/Segmented_data/MSU/label/156-180/156-180(10).wav'

# Speaker and session information
speaker = "MSU"
session = 10


for line in lines:
  parts = line.strip().split()
  start_time = float(parts[0]) / 10000
  end_time = float(parts[1]) / 10000
  sentence_number = int(parts[2])

  audio = AudioSegment.from_file(audio_file)
  extracted_audio = audio[start_time:end_time]

  output_file = f"/content/drive/Shareddrives/Audio_ processing/Segmented_data/{speaker}/speech/156-180/{speaker}_{sentence_number}_{session}.wav"
  os.makedirs(os.path.dirname(output_file), exist_ok=True)
  extracted_audio.export(output_file, format="wav")


print("done")

# Create text file

In [ ]:
# I have sentence number and transcript in a txt file eg. 1. kadxawulai wandandgu.
# @title Code to create Text file with 10 repetition for each sentence.
# @markdown ###Input file:
# @markdown - `sentenceid. \t transcript` ---> `spkid_sentenceid_repeatcount \t transcript `
# @ <speaker_sentence number_repeatcount> \t transcript each transcript should be repeated 10 times
import re

def create_spk2utt(input_file, output_file):
  """
  Processes a text file containing sentence numbers and transcripts,
  cleans the transcripts, and creates a spk2utt.txt file.

  Args:
    input_file: Path to the input text file.
    output_file: Path to the output spk2utt.txt file.
  """
  try:
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
      for line in infile:
        parts = line.strip().split('.', 1)
        if len(parts) == 2:
          spk_id = "FPU" # @param {"type":"string","placeholder":"SpeakerID eg. FPU"}
          sentence_number = parts[0].strip()
          transcript = parts[1].strip()

          # Clean the transcript
          transcript = re.sub(r'[^\w\s]', '', transcript)  # Remove punctuation
          transcript = transcript.lower()  # Convert to lowercase

          # Write to spk2utt.txt with 10 repetitions
          for i in range(1,11):
            outfile.write(f"{spk_id}_{sentence_number}_{i}\t{transcript}\n")                   # Modify Speaker ID
  except FileNotFoundError:
      print(f"Error: Input file '{input_file}' not found.")
  except Exception as e:
      print(f"An error occurred: {e}")

# Example usage (replace with your actual file paths)
input_file_path = "/content/transctipts_FPU(1).txt" # @param {"type":"string","placeholder":"Path to transcript file"}
output_file_path = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/text_all" # @param {"type":"string","placeholder":"Path to save uttrances"}
create_spk2utt(input_file_path, output_file_path)

print(f'Created text file at {output_file_path}')

In [ ]:
# @title Compare and Remove transcripts for missing wav files
# prompt: compare two file
# @markdown ###Compares wav files and remove the uncommon utt_ids and transcripts in trans display the removed items
# @markdown **Inputs  :** `wav_filepath` list of utt_id.wav and `transcript_filepath` utt_id \t transcript

def compare_and_filter(wav_file, transcript_file, output_file, removed_log_file=None):
    """
    Compares a list of utt_id.wav and a transcript file (utt_id \t transcript).
    Removes and logs transcripts whose utt_ids are not in the wav list.

    Args:
        wav_file (str): Path to the .wav list file.
        transcript_file (str): Path to the transcript file.
        output_file (str): Output file for filtered transcripts.
        removed_log_file (str, optional): Optional path to write removed items.
    """
    try:
        with open(wav_file, 'r') as wav_f:
            wav_ids = {line.strip().split('.')[0] for line in wav_f if line.strip()}

        removed_items = []

        with open(transcript_file, 'r') as trans_f, open(output_file, 'w') as out_f:
            for line in trans_f:
                if not line.strip():
                    continue
                parts = line.strip().split('\t', 1)
                if len(parts) != 2:
                    print(f"Skipping malformed line: {line.strip()}")
                    continue
                utt_id, transcript = parts
                if utt_id in wav_ids:
                    out_f.write(line)
                else:
                    removed_items.append((utt_id, transcript))

        print(f"\nTotal removed items: {len(removed_items)}")
        for idx, (utt_id, transcript) in enumerate(removed_items, start=1):
            print(f"{idx}. utt_id: {utt_id}, transcript: {transcript}")

        if removed_log_file:
            with open(removed_log_file, 'w') as log_f:
                for utt_id, transcript in removed_items:
                    log_f.write(f"{utt_id}\t{transcript}\n")

    except FileNotFoundError as fnf_err:
        print(f"File not found: {fnf_err}")
    except Exception as e:
        print(f"An error occurred: {e}")


# Example usage:
wav_filepath = "/content/wav.txt"  # @param {"type":"string","placeholder":"wavfiles list"}
transcript_filepath = "/content/text_all"  # @param{"type":"string","placeholder":"text file"}
output_filepath = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/text"  # @param{"type":"string","placeholder":"path for the output file"}
log_file = "/content/removed_items.log"  # @param{"type":"string","placeholder":"path for the log file"}
compare_and_filter(wav_filepath, transcript_filepath, output_filepath,log_file)


In [ ]:
# @title Cross valdidate  the transcripts and wavs
def find_missing_transcripts(wav_file, transcript_file, missing_output_file=None):
    """
    Identifies wav entries whose utt_ids are missing from the transcript file.

    Args:
        wav_file (str): Path to wav.txt (contains utt_id.wav)
        transcript_file (str): Path to transcript file (contains utt_id \t transcript)
        missing_output_file (str, optional): Path to save missing utt_ids. Default is None.
    """
    try:
        with open(wav_file, 'r') as wf:
            wav_ids = {line.strip().split('.')[0] for line in wf if line.strip()}

        with open(transcript_file, 'r') as tf:
            transcript_ids = {line.strip().split('\t')[0] for line in tf if '\t' in line.strip()}

        missing_utt_ids = sorted(wav_ids - transcript_ids)

        # Output
        print(f"\nTotal missing transcripts (present in wav.txt but not in transcript): {len(missing_utt_ids)}")
        for idx, utt_id in enumerate(missing_utt_ids, 1):
            print(f"{idx}. utt_id: {utt_id}")

        if missing_output_file:
            with open(missing_output_file, 'w') as out_f:
                for utt_id in missing_utt_ids:
                    out_f.write(f"{utt_id}.wav\n")  # if you want .wav extension

    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
find_missing_transcripts(
    wav_file="/content/wav.txt", # @param{"type":"string"}
    transcript_file="/content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/text", # @param{"type":"string"}
    missing_output_file="/content/missing_utt_ids.txt" # @param{"type":"string"}
)


In [ ]:
# @title Split text into Train and Test
import os

def split_data(input_file, train_output, test_output, test_ratio=0.2):
  """
  Splits the data in the input file into training and testing sets.

  Args:
    input_file: Path to the inputfile containing the data.
    train_output: Path to the output file for the training data.
    test_output: Path to the output file for the testing data.
    test_ratio: The ratio of data to be used for testing (default is 0.2)
  """
  os.makedirs(os.path.dirname(train_output), exist_ok=True)
  os.makedirs(os.path.dirname(test_output), exist_ok=True)
  try:
    with open(input_file, 'r', encoding='utf-8') as infile:
      lines = infile.readlines()
      total_lines = len(lines)
      test_size = int(total_lines * test_ratio)
      train_size = total_lines - test_size

      # Split the data into training and testing sets
      train_data = lines[:train_size]
      test_data = lines[train_size:]

      # write lines
      with open(train_output, 'w', encoding='utf-8') as train_file:
        for line in train_data:
          train_file.write(line)
      with open(test_output, 'w', encoding='utf-8') as test_file:
        for line in test_data:
          test_file.write(line)
    print("Data splitted successfully")
  except FileNotFoundError:
    print(f"Error: Input file '{input_file}' not found.")
  except Exception as e:
    print(f"An error occurred: {e}")

# Execute
input_file = "/content/text_rm" #@param{"type":"string","placeholder":"Path to spk_2 utt.txt files"}
train_output = "/content/train/text" #@param{"type":"string","placeholder":"Path to save train.txt files"}
test_output = "/content/test/text" #@param{"type":"string","placeholder":"Path to save test.txt files"}
split_data(input_file, train_output, test_output)


# Generate  utt, spk, trans, utt2spk, spk2utt files

```
asr_folder/
├── data/
│   ├── train/
│   │   ├── wav.scp
│   │   ├── spk
│   │   ├── utt
│   │   ├── spk2utt
│   │   ├── utt2spk
│   │   ├── trans
│   │   └── text
│   ├── test/
│   │   ├── wav.scp
│   │   ├── spk
│   │   ├── utt
│   │   ├── spk2utt
│   │   ├── utt2spk
│   │   ├── trans
│   │   └── text


```



In [ ]:
# @title Create spk, utt, trans files from text
def create_spk_utt_trans(input_file, output_path):
  """
  Creates spk, utt, spk2utt, trans files from a text file.

  Args:
    input_file: Path to the input text file.
    output_path: Base path for the output files.
  """

  # Get spk, utt_id, trans from text file
  with open(input_file, 'r', encoding='utf-8') as infile, open(f"{output_path}/trans",'w') as trans, open(f"{output_path}/spk",'w') as spk, open(f"{output_path}/utt",'w') as utt:
    for line in infile:
      utt_id, transcript = line.strip().split('\t')
      spk_id = utt_id.split('_')[0]

      spk.write(f"{spk_id}\n")
      utt.write(f"{utt_id}\n")
      trans.write(f"{transcript}\n")
  print("spk, utt, trans files created")

# Execute
text_input = "/content/test/text" # @param{"type":"string","placeholder":"Path to text file"}
output_path = "/content/test/" # @param{"type":"string","placeholder":"Path to save spk, utt, trans files"}
create_spk_utt_trans(text_input, output_path)

In [ ]:
# @title Create utt2spk from text

def create_utt2spk(input_file, output_file):
  """
  Processes a text file containing sentence numbers and transcripts,
  cleans the transcripts, and creates a spk2utt.txt file.
  input_file: Path to the text file.
  output_file: Path to the output spk2utt.txt file.
  """
  try:
    out_dir = os.path.dirname(output_file)
    if not os.path.exists(out_dir):
      os.makedirs(out_dir)
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
      for line in infile:
        parts = line.strip().split('\t', 1)
        if len(parts) == 2:
          utt_id = parts[0].strip()
          transcript = parts[1].strip()

          # Cut spk_Id from utt_id
          spk_id = utt_id.split('_')[0]


          # Store the spk and utt id
          outfile.write(f"{utt_id}\t{spk_id}\n")
      print("utt2spk_file created")

  except FileNotFoundError:
      print(f"Error: Input file '{input_file}' not found.")
  except Exception as e:
      print(f"An error occurred: {e}")

# Provide inputs
text_input="/content/train/text" # @param{"type":"string","placeholder":"Path to text file"}
output_file="/content/train/utt2spk" # @param{"type":"string","placeholder":"Path to save uttrances"}
create_utt2spk(text_input, output_file)

In [ ]:
# @title Create spk2utt from utt2spk

def create_spk2utt(input_file, output_file):
    """
    Creates a spk2utt file from a utt2spk file.

    Args:
        input_file: Path to the input utt2spk file (format: utt_id<tab>spk_id)
        output_file: Path to the output spk2utt file
    """
    from collections import defaultdict

    try:
        spk2utts = defaultdict(list)

        with open(input_file, 'r', encoding='utf-8') as infile:
            for line in infile:
                parts = line.strip().split('\t')
                if len(parts) != 2:
                    continue  # Skip malformed lines
                utt_id, spk_id = parts
                spk2utts[spk_id].append(utt_id)

        with open(output_file, 'w', encoding='utf-8') as outfile:
            for spk_id, utt_list in spk2utts.items():
                outfile.write(f"{spk_id} {' '.join(utt_list)}\n")

        print("spk2utt file created successfully.")
    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
# Execute
input_file = "/content/train/utt2spk" #@param{"type":"string","placeholder":"Path to spk_2 utt.txt files"}
output_file = "/content/train/spk2utt" #@param{"type":"string","placeholder":"Path to save spk2utt files"}
create_spk2utt(input_file, output_file)


In [ ]:
# @title wav.scp

def create_wav_scp(input_file, wav_dir, output_file):



# path to wav files
wav_dir ="/wavs" #@param{"type":"string","placeholder":"Path to wav files"}
# output file
output_file = "/content/wav.scp" #@param{"type":"string","placeholder":"Path to save wav.scp files"}

# Lexicon generation


>  Create ./data/local/dict/lexicon.txt

```
asr_folder/
├── data/
│   └── local/
│       └── dict/
│           └── lexicon.txt

```





In [ ]:
# @title Unique words Identification

import re
import os

def process_text_files(input_filepath, output_filepath):
    """
    Processes text files to remove unwanted symbols, split into words,
    and store unique words in a new text file.
    """
    unique_words = set()

    try:
        with open(input_filepath, 'r', encoding='utf-8') as infile:
            for line in infile:
                # Remove unwanted symbols (keep alphanumeric and spaces)
                cleaned_line = re.sub(r'[^\w\s]', '', line).lower()  # Convert to lowercase
                # Remove symbols and digits
                cleaned_line = re.sub(r'[^a-zA-Z\s]', '', line)

                # Split the line into words
                words = cleaned_line.split()

                # Add unique words to the set
                unique_words.update(words)

        # Write unique words to the output file
        with open(output_filepath, 'w', encoding='utf-8') as outfile:
            for word in sorted(unique_words):  # Sort for better readability
                outfile.write(word + '\n')

        print(f"Unique words saved to: {output_filepath}")

    except FileNotFoundError:
        print(f"Error: Input file not found at {input_filepath}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:  Replace with your actual file paths
input_file = "/content/transctipts_FPU(1).txt"                      # @param{"type":"string","placeholder": "Input file containing transcripts"}
output_file = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/unique_words.txt"                    # @param{"type":"string","placeholder":"Output file to store unique words in transcripts"}
process_text_files(input_file, output_file)

In [ ]:
# @title Create lexicon .txt

# Phonemes that should stay together
grouped_phonemes = [
    "aa", "ai", "au", "dx", "ee", "eu", "ii", "lx", "nd", "ng", "nj", "nx",
    "rx", "sx", "tx", "oo", "uu", "zh"
]

# Function to split words into phonemes
def split_into_phonemes(word, grouped_phonemes):
    phonemes = []
    i = 0
    while i < len(word):
        # Check for grouped phonemes
        matched = False
        for phoneme in grouped_phonemes:
            if word[i:i+len(phoneme)] == phoneme:
                phonemes.append(phoneme)
                i += len(phoneme)
                matched = True
                break
        # If no grouped phoneme matched, add the single character
        if not matched:
            phonemes.append(word[i])
            i += 1
    return phonemes

# Input and Output File Paths
input_file = "/content/unique_words.txt"                       # @param{"type":"string","placeholder": "Input file containing words"}
output_file = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Punitha_FPU/lexicon.txt"                           # @param{"type":"string","placeholder":"Output file to store word-phoneme mappings"}

# Read words from the input file
with open(input_file, "r") as infile:
    words = [line.strip() for line in infile.readlines()]

# Generate phoneme sequences for each word
results = []
for word in words:
    phoneme_sequence = split_into_phonemes(word, grouped_phonemes)
    results.append(f"{word}\t{' '.join(phoneme_sequence)}")

# Write the results to the output file
with open(output_file, "w") as outfile:
    for line in results:
        outfile.write(line + "\n")

print(f"Phoneme sequences written to {output_file}")


In [ ]:
# @title Merge lexicons

# prompt: i have file 1 and file 2 .txt i need to compare  words in file 1 with the 1st word in file 2 and to display the unmatches words in file 1

def compare_words(file1_path, file2_path):
    """
    Compares words in file1 with the first word in file2 and displays unmatched words from file1.
    """
    try:
        with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
            file1_words = set()
            for line in file1:
                words = line.strip().split()  # Split each line into words
                file1_words.update(words)

            first_word_file2 = file2.readline().strip().split()[0] # Get only the 1st word in file2

            unmatched_words = file1_words - set([first_word_file2]) #difference between file1 and file2

            for word in unmatched_words:
                print(word)

    except FileNotFoundError:
        print(f"Error: One or both of the files were not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


# Example usage: Replace with your actual file paths
file1_path = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/MSU/unique_words.txt"
file2_path = "/content/drive/MyDrive/lexicon.txt"
compare_words(file1_path, file2_path)

# Audio normalization

In [ ]:
import os
from pydub import AudioSegment
from pydub.effects import normalize

def normalize_audio_files(input_dir, output_dir):
    """
    Normalize audio files, including those in subfolders.

    Args:
        input_dir (str): Directory containing input audio files (including subfolders).
        output_dir (str): Directory to save normalized audio files, maintaining folder structure.
    """
    for root, dirs, files in os.walk(input_dir):  # Traverse all subdirectories
        for filename in files:
            if filename.endswith((".wav", ".mp3", ".flac", ".ogg")):  # Add other formats if needed
                input_path = os.path.join(root, filename)

                # Recreate the subfolder structure in the output directory
                relative_path = os.path.relpath(root, input_dir)
                output_subdir = os.path.join(output_dir, relative_path)
                if not os.path.exists(output_subdir):
                    os.makedirs(output_subdir)

                output_path = os.path.join(output_subdir, filename)

                try:
                    print(f"Processing: {input_path}")

                    # Load the audio file
                    audio = AudioSegment.from_file(input_path)

                    # Normalize the audio
                    normalized_audio = normalize(audio)

                    # Export the normalized audio to the output directory
                    normalized_audio.export(output_path, format="wav")  # Ensure output is in `.wav` format
                    print(f"Saved normalized file to: {output_path}")

                except Exception as e:
                    print(f"Error processing {input_path}: {e}")

# Set input and output directories
input_directory = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech"
output_directory = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech_normalised"

# Run the normalization
normalize_audio_files(input_directory, output_directory)


In [ ]:
# prompt: i have files in sub dirs such as MSU_1_1.wav, MSU_2_1.wav so on i need to rename it as MSR_1_1.wav , MSR_2_1.wav..

import os
import re

def rename_files(root_dir):
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.startswith("MSU_") and file.endswith(".wav"):
                old_path = os.path.join(subdir, file)
                new_file = file.replace("MSU", "MSR", 1)
                new_path = os.path.join(subdir, new_file)
                os.rename(old_path, new_path)
                # print(f"Renamed '{old_path}' to '{new_path}'")
        print("All files renamed successfully.")

# Example usage (replace with your actual directory)
root_directory = "/content/drive/Shareddrives/Audio_ processing/Segmented_data/Suradha_MSR/speech_normalised/"
rename_files(root_directory)

In [ ]:
# @title Prompt file generation
def prompt_generation(input_file, output_path):
  with open(input_file, 'r', encoding='utf-8') as infile:
    for line in infile:
      utt_id, transcript = line.strip().split('\t')
      with open(os.path.join(output_path, f"{utt_id}.txt"), 'w', encoding='utf-8') as outfile:
        outfile.write(line)
    print("Prompt files generated")
    except FileNotFoundError:
    print(f"Error: Input file '{input_file}' not found.")
  except Exception as e:
    print(f"An error occurred: {e}")


# execution
input_file = "/content/train/text" #@param{"type":"string","placeholder":"Path to text file"}
output_path = "/content/train/prompts" #@param{"type":"string","placeholder":"Path to save prompt files"}
prompt_generation(input_file, output_path)
#